In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!python3.10 --version > /dev/null 2>&1
!sudo apt-get update -qq > /dev/null 2>&1
!sudo apt-get install -y python3.10-distutils -qq > /dev/null 2>&1
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py > /dev/null 2>&1
!python3.10 get-pip.py > /dev/null 2>&1


In [2]:
!python3.10 -m pip install -U pip setuptools wheel > /dev/null 2>&1

In [3]:
!rm -rf TTS && \
git clone --branch add-vietnamese-xtts https://github.com/thinhlpg/TTS.git && \
cd TTS && \
python3.10 -m pip install --use-deprecated=legacy-resolver --ignore-installed -e . && \
python3.10 -m pip install -U "numpy>=1.23.5,<2.0" scipy matplotlib librosa contourpy umap-learn "networkx>=2.5,<3.0" > /dev/null 2>&1


Cloning into 'TTS'...
remote: Enumerating objects: 28904, done.
remote: Total 28904 (delta 0), reused 0 (delta 0), pack-reused 28904 (from 1)
Receiving objects: 100% (28904/28904), 136.14 MiB | 16.26 MiB/s, done.
Resolving deltas: 100% (20953/20953), done.
Obtaining file:///content/TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 38.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 78.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 51.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 15.5 MB/s  0:00:22
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!python3.10 -m pip uninstall -y numpy pandas
!python3.10 -m pip install \
  numpy==1.23.5 \
  pandas==1.5.3 > /dev/null 2>&1


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 1.5.3
Uninstalling pandas-1.5.3:
  Successfully uninstalled pandas-1.5.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 34.7 MB/s  0:00:00
Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.23.5 which is incompatible.


In [5]:
!python3.10 -m pip install torchcodec > /dev/null 2>&1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.5 MB/s  0:00:00


In [6]:
!python3.10 -m pip install openai-whisper > /dev/null 2>&1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 7.4 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s  0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=9c65e27404fb3fe94c4342e077c11ff6271a0ac601e147b8cacc4eae43222f2c
  Stored in directory: /root/.cache/pip/wheels/be/22/86/dcf050d50e02b3bb67b5a3b289be2f3376ce7f8717d012d6a8
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openai-whisper]


# viXTTS Quick Usage




In [7]:
%%writefile run_xtts.py
import os
import glob
import argparse
import torch
import numpy as np
import whisper
from huggingface_hub import snapshot_download
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

MODEL_DIR = "model"
SAMPLE_RATE = 24000


def load_xtts_model():
    """Load XTTS model"""
    # download model (cache, không tải lại nếu đã có)
    snapshot_download(
        repo_id="capleaf/viXTTS",
        repo_type="model",
        local_dir=MODEL_DIR,
        local_dir_use_symlinks=False,
    )

    # load config
    config = XttsConfig()
    config.load_json(f"{MODEL_DIR}/config.json")

    # init model
    model = Xtts.init_from_config(config)
    model.load_checkpoint(config, checkpoint_dir=MODEL_DIR)
    model.eval()

    if torch.cuda.is_available():
        model.cuda()

    # conditioning (voice reference)
    gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(
        audio_path=f"{MODEL_DIR}/vi_sample.wav",
        gpt_cond_len=model.config.gpt_cond_len,
        max_ref_length=model.config.max_ref_len,
        sound_norm_refs=model.config.sound_norm_refs,
    )

    return model, gpt_cond_latent, speaker_embedding


def xtts_inference(model, text, gpt_cond_latent, speaker_embedding):
    """Run XTTS inference, return audio numpy array"""
    out = model.inference(
        text=text,
        language="vi",
        gpt_cond_latent=gpt_cond_latent,
        speaker_embedding=speaker_embedding,
        temperature=0.3,
        length_penalty=1.0,
        repetition_penalty=10.0,
        top_k=30,
        top_p=0.85,
    )
    return np.array(out["wav"], dtype=np.float32)


def process_folder(input_dir, output_dir, num_files=None):
    """
    Process all .txt files in input_dir
    - XTTS: text -> audio (in memory)
    - Whisper: audio -> transcript
    - Save to output_dir
    """
    os.makedirs(output_dir, exist_ok=True)

    # Get all txt files, sorted
    txt_files = sorted(glob.glob(os.path.join(input_dir, "*.txt")))

    if num_files:
        txt_files = txt_files[:num_files]

    print(f"Found {len(txt_files)} files to process")

    # Load models (once)
    print("Loading XTTS model...")
    xtts_model, gpt_cond_latent, speaker_embedding = load_xtts_model()

    print("Loading Whisper model...")
    whisper_model = whisper.load_model("base")

    # Process each file
    for i, txt_file in enumerate(txt_files):
        filename = os.path.basename(txt_file)

        # 1. Read text
        with open(txt_file, 'r', encoding='utf-8') as f:
            text = f.read().strip()

        if not text:
            print(f"[{i+1}/{len(txt_files)}] {filename} - SKIPPED (empty)")
            continue

        # 2. XTTS inference -> audio array
        audio = xtts_inference(xtts_model, text, gpt_cond_latent, speaker_embedding)

        # 3. Whisper transcribe (from numpy array)
        # Whisper expects float32 audio at 16kHz, but can handle other sample rates
        result = whisper_model.transcribe(
            audio,
            language="vi",
            fp16=torch.cuda.is_available()
        )
        transcript = result["text"].strip()

        # 4. Save transcript
        output_file = os.path.join(output_dir, filename)
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(transcript)

        print(f"[{i+1}/{len(txt_files)}] {filename} - OK")

    print(f"\n✓ Done! Results saved to {output_dir}/")


def main():
    parser = argparse.ArgumentParser(description='XTTS -> Whisper Pipeline')
    parser.add_argument('-i', '--input', default='ground-truth',
                        help='Input folder with .txt files (default: ground-truth)')
    parser.add_argument('-o', '--output', default='xtts/text',
                        help='Output folder for transcripts (default: xtts/text)')
    parser.add_argument('-n', '--num-files', type=int,
                        help='Number of files to process (default: all)')

    args = parser.parse_args()
    process_folder(args.input, args.output, args.num_files)


if __name__ == "__main__":
    main()

Writing run_xtts.py


In [ ]:
# Run pipeline: ground-truth -> XTTS -> Whisper -> xtts/text
# Sử dụng -n để giới hạn số file (vd: -n 10 để test 10 file đầu)
!MPLBACKEND=Agg python3.10 run_xtts.py --input content/drive/ground-truth --output xtts/text

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 16 files: 1